In [ ]:
import os
import re
from tqdm import tqdm
import pandas as pd

def extraire_date(nom_fichier):
    match = re.match(r"(\d{4}-\d{2}-\d{2})", nom_fichier)
    if match:
        return match.group(1)
    return None

def concordancier(dossier, mot_cible, contexte, extrait=1000):
    lignes_df = []  

    for fichier in tqdm(os.listdir(dossier)):
        if fichier.endswith('.txt') and fichier != "french_stopwords.txt":
            date_document = extraire_date(fichier)

            if date_document:
                with open(os.path.join(dossier, fichier), 'r', encoding='utf-8') as f:
                    longue_chaine = f.read()

                mots = re.findall(r'\b\w+\b', longue_chaine.lower())
                indices = [i for i, mot in enumerate(mots) if mot_cible.lower() in mot]
                
                for index in indices:
                    debut = max(0, index - extrait)  
                    fin = min(len(mots), index + extrait + 1)  
                    extrait_complet = mots[debut:fin]
                    texte_total = ' '.join(extrait_complet)

                    contexte_avant = mots[max(0, index - contexte):index]
                    contexte_apres = mots[index + 1:min(len(mots), index + 1 + contexte)]

                    lignes_df.append({
                        'date': date_document,
                        'contexte_avant': ' '.join(contexte_avant),
                        'mot_cible': mot_cible,
                        'contexte_apres': ' '.join(contexte_apres),
                        'texte_total': texte_total
                    })
    df_concord = pd.DataFrame(lignes_df)
    return df_concord

# ============================
dossier = 'C:/Users/portable_laura/Documents/coursM1/Mémoire/Sources/ocr_sorted'
mot_cible = "barrage"
contexte = 8  
longueur_extrait = 500 

concordances = concordancier(dossier, mot_cible, contexte, extrait=longueur_extrait)
concordances.head()
dates = concordances['date'].unique()
dates_liste = dates.tolist()
print(f'{len(dates_liste)} séances mentionnent le terme "{mot_cible}"')

concordances.to_csv(f"C:/Users/portable_laura/Desktop/memoire_prod_finale/data/{mot_cible}.csv", index=False, encoding='utf-8')
#=====================